<a href="https://colab.research.google.com/github/tnflynt/Data_Analyst_Portfolio/blob/main/Heart_disease_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Heart disease prediction



### The objective of this exercise is to buid a machine learning model to accurately predict whether a patient has a heart disease or not (i.e. Outcome = 1 ~ is at risk of heart disease and Outcome = 0 ~ is no risk of heart disease)

**Setting up PySpark and creating a Spark Session**

In [ ]:
!sudo apt update
!sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
15 packages can be upgraded. Run 

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive"

 airline.csv	        cal_housing.data		      diabetes.csv
 bank.csv	        cal_housing.domain		      heart.csv
 boston_data.csv       'Colab Notebooks'
 boston_test_data.csv   Copy_of_Introduction_to_Spark.ipynb


In [ ]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession

# Create spark_session
spark_session = SparkSession.builder.getOrCreate()

In [ ]:
df = spark_session.read.option("header", "true").csv("/content/drive/My Drive/heart.csv")

In [ ]:
df.show()
df.dtypes

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|
| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|
| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|     1|
| 56|  1|  1|     120| 236|  0|      1|    178|    0|    0.8|    2|  0|   2|     1|
| 57|  0|  0|     120| 354|  0|      1|    163|    1|    0.6|    2|  0|   2|     1|
| 57|  1|  0|     140| 192|  0|      1|    148|    0|    0.4|    1|  0|   1|     1|
| 56|  0|  1|     140| 294|  0|      0|    153|    0|    1.3|    1|  0|   2|     1|
| 44|  1|  1|     120| 263|  0|      1|    173|    0|      0|    2|  0|   3|     1|
| 52|  1|  2|     172| 199|  1|      1|    162|    0|    0.5|    2|  0|   3|

[('age', 'string'),
 ('sex', 'string'),
 ('cp', 'string'),
 ('trestbps', 'string'),
 ('chol', 'string'),
 ('fbs', 'string'),
 ('restecg', 'string'),
 ('thalach', 'string'),
 ('exang', 'string'),
 ('oldpeak', 'string'),
 ('slope', 'string'),
 ('ca', 'string'),
 ('thal', 'string'),
 ('target', 'string')]

In [ ]:
df.createOrReplaceTempView("heart")

In [ ]:
modeldata = spark_session.sql("SELECT * FROM heart")
modeldata.show()

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|
| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|
| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|     1|
| 56|  1|  1|     120| 236|  0|      1|    178|    0|    0.8|    2|  0|   2|     1|
| 57|  0|  0|     120| 354|  0|      1|    163|    1|    0.6|    2|  0|   2|     1|
| 57|  1|  0|     140| 192|  0|      1|    148|    0|    0.4|    1|  0|   1|     1|
| 56|  0|  1|     140| 294|  0|      0|    153|    0|    1.3|    1|  0|   2|     1|
| 44|  1|  1|     120| 263|  0|      1|    173|    0|      0|    2|  0|   3|     1|
| 52|  1|  2|     172| 199|  1|      1|    162|    0|    0.5|    2|  0|   3|

Cast the columns into appropriate data types

In [ ]:
# Cast the columns to an appropriate data type
modeldata = modeldata.withColumn("age", modeldata.age.cast("integer"))
modeldata = modeldata.withColumn("trestbps", modeldata.trestbps.cast("integer"))
modeldata = modeldata.withColumn("chol", modeldata.chol.cast("integer"))
modeldata = modeldata.withColumn("thalach", modeldata.thalach.cast("integer"))
modeldata = modeldata.withColumn("oldpeak", modeldata.oldpeak.cast("float"))
modeldata = modeldata.withColumn("target", modeldata.target.cast("integer"))

modeldata.show()
modeldata.dtypes

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|
| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|
| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|     1|
| 56|  1|  1|     120| 236|  0|      1|    178|    0|    0.8|    2|  0|   2|     1|
| 57|  0|  0|     120| 354|  0|      1|    163|    1|    0.6|    2|  0|   2|     1|
| 57|  1|  0|     140| 192|  0|      1|    148|    0|    0.4|    1|  0|   1|     1|
| 56|  0|  1|     140| 294|  0|      0|    153|    0|    1.3|    1|  0|   2|     1|
| 44|  1|  1|     120| 263|  0|      1|    173|    0|    0.0|    2|  0|   3|     1|
| 52|  1|  2|     172| 199|  1|      1|    162|    0|    0.5|    2|  0|   3|

[('age', 'int'),
 ('sex', 'string'),
 ('cp', 'string'),
 ('trestbps', 'int'),
 ('chol', 'int'),
 ('fbs', 'string'),
 ('restecg', 'string'),
 ('thalach', 'int'),
 ('exang', 'string'),
 ('oldpeak', 'float'),
 ('slope', 'string'),
 ('ca', 'string'),
 ('thal', 'string'),
 ('target', 'int')]

Examine the data

In [ ]:
modeldata.describe().select('summary','age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal').show()

+-------+------------------+-------------------+------------------+------------------+------------------+-------------------+-----------------+------------------+-------------------+------------------+------------------+------------------+------------------+
|summary|               age|                sex|                cp|          trestbps|              chol|                fbs|          restecg|           thalach|              exang|           oldpeak|             slope|                ca|              thal|
+-------+------------------+-------------------+------------------+------------------+------------------+-------------------+-----------------+------------------+-------------------+------------------+------------------+------------------+------------------+
|  count|               303|                303|               303|               303|               303|                303|              303|               303|                303|               303|               303|   

Use "StringIndexer" and "OneHotEncoding" to convert categorical features into numeric values. Combine all of the columns containing our features for the model into a single column - "target" is the label that the model will predict.

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler

#Store all the categorical columns
categorical_Cols = ['sex','cp','fbs','restecg','exang','slope','ca','thal']
pipeline_stages = []

#Iterate through the categorical columns
for categoricalCol in categorical_Cols:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + '_Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "_conv"])
    pipeline_stages += [stringIndexer, encoder]

#Store all the numeric columns
numericCols = ['age','trestbps','chol','thalach','oldpeak']

#Combine all of the columns containing our features for the model into a single column
assembler_Inputs = [c + "_conv" for c in categorical_Cols] + numericCols
assembler = VectorAssembler(inputCols=assembler_Inputs, outputCol="features")
pipeline_stages += [assembler]

In [ ]:
# Create label column
from pyspark.sql.functions import *
modeldata = modeldata.withColumn("label", col("target"))

We use the ML Pipeline to chain multiple Transformers and Estimators together to screate the machine learning workflow.

In [ ]:
from pyspark.ml import Pipeline

ml_pipe = Pipeline(stages = pipeline_stages)
# Fit and transform the data
pipe_data = ml_pipe.fit(modeldata).transform(modeldata)
selectedCols = ['label','features','age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','target']
modeldata = pipe_data.select(selectedCols)
modeldata.show()

+-----+--------------------+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|label|            features|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+-----+--------------------+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|    1|(22,[0,6,7,10,16,...| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|
|    1|(22,[0,2,4,5,7,10...| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|
|    1|(22,[3,4,6,7,8,10...| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|     1|
|    1|(22,[0,3,4,5,7,8,...| 56|  1|  1|     120| 236|  0|      1|    178|    0|    0.8|    2|  0|   2|     1|
|    1|(22,[1,4,5,8,10,1...| 57|  0|  0|     120| 354|  0|      1|    163|    1|    0.6|    2|  0|   2|     1|
|    1|(22,[0,1,4,5,7,9,...| 57|  1|  0|     140| 192|  0|      1|    148|    0|    0.4|    1|  0|   1|     1|
|

In [ ]:
train, test = modeldata.randomSplit([0.8, 0.2], seed=12345)
train.show(20)

+-----+--------------------+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|label|            features|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+-----+--------------------+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|    0|(22,[0,1,4,5,7,8,...| 40|  1|  0|     152| 223|  0|      1|    181|    0|    0.0|    2|  0|   3|     0|
|    0|(22,[0,1,4,5,7,8,...| 52|  1|  0|     112| 230|  0|      1|    160|    0|    0.0|    2|  1|   2|     0|
|    0|(22,[0,1,4,5,7,8,...| 61|  1|  0|     148| 203|  0|      1|    161|    0|    0.0|    2|  1|   3|     0|
|    0|(22,[0,1,4,5,7,8,...| 58|  1|  0|     100| 234|  0|      1|    156|    0|    0.1|    2|  1|   3|     0|
|    0|(22,[0,1,4,5,7,8,...| 52|  1|  0|     125| 212|  0|      1|    168|    0|    1.0|    2|  2|   3|     0|
|    0|(22,[0,1,4,5,7,9,...| 67|  1|  0|     120| 237|  0|      1|     71|    0|    1.0|    1|  0|   2|     0|
|

After the train/test split, it is a good practice to check if there is a imbalance in the train dataset. 

In [ ]:
size_of_dataset = float(train.select("target").count())
no_of_Positives = train.select("target").where('target == 1').count()
percent_ones=(float(no_of_Positives)/float(size_of_dataset))*100
no_of_Negatives=float(size_of_dataset - no_of_Positives)
print('The number of ones are {}'.format(no_of_Positives))
print('Percentage of ones are {}'.format(percent_ones))

The number of ones are 134
Percentage of ones are 53.17460317460318


Conclusion: The training dataset is quite evenly balanced

**Selecting Features**

The underlying data set has several features; however, 
some of these features are more signifcant as compared to others. Therefrore, we will leverage the "ChiSqSelector" class provided by Spark ML for selecting the significant features.

In [ ]:
from pyspark.ml.feature import ChiSqSelector

selector = ChiSqSelector(featuresCol='features',outputCol='Significant_feat',labelCol='target',fpr=0.05)
train=selector.fit(train).transform(train)
test=selector.fit(test).transform(test)
test.select("Significant_feat").show(20,truncate=False)



+-------------------------------------------------------------------------------------------------------------------+
|Significant_feat                                                                                                   |
+-------------------------------------------------------------------------------------------------------------------+
|(22,[0,1,4,5,7,9,10,15,17,18,19,20,21],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,39.0,118.0,219.0,140.0,1.2000000476837158])|
|(22,[0,1,4,5,9,13,15,17,18,19,20,21],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,51.0,140.0,298.0,122.0,4.199999809265137])       |
|(22,[0,1,4,5,10,15,17,18,19,20,21],[1.0,1.0,1.0,1.0,1.0,1.0,55.0,140.0,217.0,111.0,5.599999904632568])             |
|(22,[0,1,4,6,7,8,10,15,17,18,19,20,21],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,46.0,120.0,249.0,144.0,0.800000011920929]) |
|(22,[0,1,4,6,7,9,11,15,17,18,19,20,21],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,63.0,130.0,254.0,147.0,1.399999976158142]) |
|(22,[0,1,4,6,9,10,15,17,18,19,20,21],[1.0,1.0,1.0,1.0,1

**Creating the Logistic Regression Model**

In [ ]:
from pyspark.ml.classification import LogisticRegression

logit_regression = LogisticRegression(labelCol="target", featuresCol="Significant_feat",maxIter=25)
model=logit_regression.fit(train)
prediction_on_train_dataset=model.transform(train)
prediction_on_test_dataset=model.transform(test)
prediction_on_test_dataset.select("target","rawPrediction","probability","prediction").show(30)

+------+--------------------+--------------------+----------+
|target|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+----------+
|     0|[1.80277928936483...|[0.85848691951327...|       0.0|
|     0|[6.95022734344597...|[0.99904250044503...|       0.0|
|     0|[4.68485906921245...|[0.99085045137976...|       0.0|
|     0|[0.83428759431574...|[0.69726075510254...|       0.0|
|     0|[4.63608711464237...|[0.99039754011609...|       0.0|
|     0|[2.52752726516890...|[0.92604919365878...|       0.0|
|     0|[3.18982857338739...|[0.96044970914051...|       0.0|
|     0|[4.66065232875761...|[0.99062836916790...|       0.0|
|     0|[-1.6602202781472...|[0.15973242943953...|       1.0|
|     0|[1.98692673656585...|[0.87941762153031...|       0.0|
|     0|[1.20507209909940...|[0.76942585255753...|       0.0|
|     0|[1.00255542261577...|[0.73156070845377...|       0.0|
|     0|[-1.4965112109885...|[0.18294644093137...|       1.0|
|     0|

**Evaluating the Model** - Since the Logistic model at hand is a binary classification model (given that the outcome is 1 or 0), the BinaryClassificationEvaluator from the ***pyspark.ml.evaluation*** module will be utilized to evaluate the Logistic Regression Model.

The BinaryClassificationEvaluator calculates the area under the ROC which is is one of the most important evaluation metrics for checking any classification model’s performance. It finds the best model by maximizing the model evaluation metric that is the area under the specified curve. The closer the area Under Curve is to one (1), the better the model is!


In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol="target",metricName="areaUnderROC")

# Evaluate the predictions
print("The area under ROC for test data set is {}".format(evaluator.evaluate(prediction_on_test_dataset)))

The area under ROC for test data set is 0.896774193548387


**The area under ROC is quite large, therefore, it can be concluded that the model generated is good in predicting the onset of heart disease.**